# Set-up SQLite Database

In [1]:
import sqlite3
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.35.4


In [2]:
product = pd.read_csv('../data/product.csv')
product_price = pd.read_csv('../data/product_price.csv')
vendor = pd.read_csv('../data/vendor.csv')
store = pd.read_csv('../data/store.csv')
transaction = pd.read_csv('../data/transaction.csv')

/Users/flyingcape/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/flyingcape/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
dbconn = sqlite3.connect('../data/IowaLiquorSales.db') # database
cursor = dbconn.cursor() # create a cursor to use to execute sql statements

In [4]:
# update to include final columns for each table
cursor.execute('''CREATE TABLE Product
                (ProductID, ProductDescription, CategoryCode, CategoryName, CategoryGroup, Pack, BottleVolume, Proof, VendorID)''')

cursor.execute('''CREATE TABLE Price
                (ProductID, Date, StateBottleRetail, StateBottleCost)''')

cursor.execute('''CREATE TABLE Vendor
                (VendorID, VendorName)''')

cursor.execute('''CREATE TABLE Store
                (StoreNumber, StoreName, Address, City, ZipCode, StoreLocation, CountyNumber, County)''')

cursor.execute('''CREATE TABLE Transactions
                (ItemNumber, Date, StoreNumber, ProductNumber, VendorNumber, Category, BottlesSold, VolumeSold, Sale)''')

In [5]:
product.to_sql('Product', dbconn, if_exists = 'append', index = False)
product_price.to_sql('Price', dbconn, if_exists = 'append', index = False)
vendor.to_sql('Vendor', dbconn, if_exists = 'append', index = False)
store.to_sql('Store', dbconn, if_exists = 'append', index = False)
transaction.to_sql('Transactions', dbconn, if_exists = 'append', index = False)

In [7]:
dbconn.commit()

In [9]:
dbconn.close()